In [ ]:
import csv
import pandas

In [ ]:
rec_data_name = 'data/recievers.csv'
rec_data = pandas.read_csv(rec_data_name)



In [ ]:
m = folium.Map(location=[57.0890, -4.7483], zoom_start=15)
for index, rec in rec_data[rec_data['system']=='Ness'].iterrows():
    folium.Marker([rec['lat'], rec['lon']], popup=rec['id']).add_to(m)

In [ ]:
m